In [2]:
import torch
import transformers
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

In [3]:
# Load the dataset
datasets = load_dataset("squad_v2")
print(datasets)
print("Context: ", datasets["train"][0]["context"])
print("Question: ", datasets["train"][0]["question"])
print("Answers: ", datasets["train"][0]["answers"])

Using the latest cached version of the module from C:\Users\suiyu\.cache\huggingface\modules\datasets_modules\datasets\squad_v2\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d (last modified on Tue Apr 19 00:19:21 2022) since it couldn't be found locally at squad_v2., or remotely on the Hugging Face Hub.
Reusing dataset squad_v2 (C:\Users\suiyu\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})
Context:  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Question:  When did Beyonce st

In [4]:
# Dataset.filter(), during training, there is only one possible answer.
# However, for the evaluation, there are several possible answers for each sample, which may be the same or different.
datasets["train"].filter(lambda x:len(x["answers"]["text"]) != 1)

Loading cached processed dataset at C:\Users\suiyu\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d\cache-de60272c773bd99f.arrow


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 43498
})

In [8]:
# Preprocessing the training data
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Pass to our tokenizer the question and the context together
context = datasets["train"][0]["context"]
question = datasets["train"][0]["question"]

In [9]:
# not using in the following sections (only for testing)
model_inputs = tokenizer(question, context)
# [CLS] question [SEP] context [SEP]
print(model_inputs)
tokenizer.decode(model_inputs["input_ids"])

{'input_ids': [101, 2043, 2106, 20773, 2707, 3352, 2759, 1029, 102, 20773, 21025, 19358, 22815, 1011, 5708, 1006, 1013, 12170, 23432, 29715, 3501, 29678, 12325, 29685, 1013, 10506, 1011, 10930, 2078, 1011, 2360, 1007, 1006, 2141, 2244, 1018, 1010, 3261, 1007, 2003, 2019, 2137, 3220, 1010, 6009, 1010, 2501, 3135, 1998, 3883, 1012, 2141, 1998, 2992, 1999, 5395, 1010, 3146, 1010, 2016, 2864, 1999, 2536, 4823, 1998, 5613, 6479, 2004, 1037, 2775, 1010, 1998, 3123, 2000, 4476, 1999, 1996, 2397, 4134, 2004, 2599, 3220, 1997, 1054, 1004, 1038, 2611, 1011, 2177, 10461, 1005, 1055, 2775, 1012, 3266, 2011, 2014, 2269, 1010, 25436, 22815, 1010, 1996, 2177, 2150, 2028, 1997, 1996, 2088, 1005, 1055, 2190, 1011, 4855, 2611, 2967, 1997, 2035, 2051, 1012, 2037, 14221, 2387, 1996, 2713, 1997, 20773, 1005, 1055, 2834, 2201, 1010, 20754, 1999, 2293, 1006, 2494, 1007, 1010, 2029, 2511, 2014, 2004, 1037, 3948, 3063, 4969, 1010, 3687, 2274, 8922, 2982, 1998, 2956, 1996, 4908, 2980, 2531, 2193, 1011, 2028, 38

'[CLS] when did beyonce start becoming popular? [SEP] beyonce giselle knowles - carter ( / biːˈjɒnseɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny\'s child. managed by her father, mathew knowles, the group became one of the world\'s best - selling girl groups of all time. their hiatus saw the release of beyonce\'s debut album, dangerously in love ( 2003 ), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number - one singles " crazy in love " and " baby boy ". [SEP]'

In [14]:
# Dealing with the long contexts by creating several training features from one sample of our dataset, with a sliding window between them.
# - max_length, set the maximum length (here 100)
# - truncation="only_second", truncate the context (which is the second position)
# - stride, set the number of overlapping tokens between two successive chunks (here 50)
# - return_overflowing_tokens=True, to let the tokenizer know we want the overflowing tokens.

model_inputs = tokenizer(
    question, context,
    max_length = 100, # if the context len > 100, it will be silded into several parts.
    truncation = "only_second",
    stride = 50,
    return_overflowing_tokens = True,
    return_offsets_mapping = True
)

for ids in model_inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] when did beyonce start becoming popular? [SEP] beyonce giselle knowles - carter ( / biːˈjɒnseɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny's child. managed by her father, [SEP]
[CLS] when did beyonce start becoming popular? [SEP] actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny's child. managed by her father, mathew knowles, the group became one of the world's best - selling girl groups of all time. their hiatus saw the release of beyonce's debut album, dangerously in love ( 2003 ), [SEP]
[CLS] when did beyonce start becoming popular? [SEP] destiny's child. managed by her father, mathew kno

In [15]:
model_inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [16]:
model_inputs["overflow_to_sample_mapping"]

[0, 0, 0]

In [22]:
model_inputs = tokenizer(
    datasets["train"][2:6]["question"],
    datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True, # offset 偏移量
)

print(f"The 4 examples gave {len(model_inputs['input_ids'])} features.")
print(f"Here is where each comes from: {model_inputs['overflow_to_sample_mapping']}.")
print(len(model_inputs["offset_mapping"])) # len=15

The 4 examples gave 15 features.
Here is where each comes from: [0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3].
15


In [30]:
answers = datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(model_inputs["offset_mapping"]):
    sample_idx = model_inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    sequence_ids = model_inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([0, 0, 77, 44, 58, 21, 0, 0, 78, 39, 0, 94, 59, 24, 0],
 [0, 0, 77, 44, 60, 23, 0, 0, 79, 40, 0, 97, 62, 27, 0])

In [34]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [40]:
train_dataset = datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=datasets["train"].column_names,
)
len(datasets["train"]), len(train_dataset)

  0%|          | 0/131 [00:00<?, ?ba/s]

IndexError: list index out of range

In [38]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [39]:
validation_dataset = datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)
len(datasets["validation"]), len(validation_dataset)

  0%|          | 0/12 [00:00<?, ?ba/s]

(11873, 12134)